# 04 - Querying data tables with Pandas

This tutorial covers database-like features of the Pandas module.  It adds new objects: Dataframes, which are tables of data, and Series, which represent a single column or row.  They provide equivalent functions to the R programming language.  They can be used to analyse tables of data within Python using similar operations to those provided by Structured Query Language ([SQL](https://en.wikipedia.org/wiki/SQL)), which is used with relational database management systems.

Here, we will query the [Global Volcanism Program](http://www.volcano.si.edu/list_volcano_holocene.cfm)'s spreadsheet of Holocene volcanoes.  Many of the queries are based on the _Generate volcano trivia with this SQLite tutorial_ blog [post](http://www.volcano.si.edu/list_volcano_holocene.cfm). 

### Setting up

In [ ]:
# Import modules
import datetime as dt
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Show plots within notebooks
%matplotlib inline

# Show module versions
print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))
from matplotlib import __version__ as mplv
print('Matplotlib: {}'.format(mplv))

# Setup working directory
wdir = os.getcwd()  # Change this if required
data_dir = os.path.join(wdir, 'data_files')
os.chdir(wdir)

### Import and check the data

In [ ]:
# Pandas' read_csv command has lots of options for dealing with headers, footers etc.
gvp = pd.read_csv(os.path.join(data_dir, 'GVP_Volcano_List.csv'))

In [ ]:
# Check the column names.
print(gvp.columns)

In [ ]:
# View the first 5 entries
gvp.head()

In [ ]:
# View the last 2 entries
gvp.tail(2)

### Accessing data

In [ ]:
# Access individual columns as Pandas series using Python dictionary syntax.
gvp['Last Known Eruption'].tail()

In [ ]:
#  Get summary statistics for a column
print(gvp['Elevation (m)'].describe())

In [ ]:
# Data rows can be sliced based on their position
gvp[150:155]

In [ ]:
# Using the index and the '.loc' command gives more meaningful access to the rows
gvp_names = gvp.set_index('Volcano Name')  # Make a copy with the Name column as the index
gvp_names.loc['Hekla']  # What information is there on Hekla?

In [ ]:
# The '.loc' command accepts lists of rows and columns
gvp_names.loc[['Fuji', 'Etna'], 'Elevation (m)']  # Which is taller, Fuji or Etna?

In [ ]:
#  True/False (Boolean) indexing can be used to perform SQL-like 'WHERE' queries.
gvp_names.loc[gvp_names['Country'] == 'Iceland', ['Latitude', 'Longitude']]  # Where are Iceland's volcanoes?

### Manipulating data within the columns

In [ ]:
# Add extra columns using dictionary assignment notation
gvp_names['Elevation (km)'] = gvp_names['Elevation (m)']/1000.0
gvp_names['Elevation (km)'].head()

In [ ]:
#  Operations on Series that contains text strings are done with the .str.command syntax
gvp_regions = gvp['Region']
print(gvp_regions.str.upper().head())  # Make text ALL CAPS
print()
print(gvp_regions.str.replace('and', '&').head())  # find and replace
print()
print(gvp_regions.str.contains('Asia').head())  # Search the text

In [ ]:
#  Use the .map method to apply a function to every entry in a column

# Define a function to convert text year into numerical value
def extract_year(year_string):
    """Drop the CE or BCE suffix from a date, e.g. '2015 CE', returning an integer.
    This allows sorting and indexing by the numerical value of the year."""
    if year_string == 'Unknown':
        return None
    
    year = year_string.split()[0]
    suffix = year_string.split()[1]
    
    if suffix == 'CE':
        return int(year)
    elif suffix == 'BCE':
        return -1 * int(year)
    else:
        raise ValueError('Cannot parse {} as eruption year.'.format(year_string))

# Add numerical date column and analyse
gvp['Last Known Eruption Year'] = gvp['Last Known Eruption'].map(extract_year)  # Applying functions to change values
gvp_years = gvp.dropna(subset=['Last Known Eruption Year']).set_index('Last Known Eruption Year').sort_index()
gvp_years.loc[1914:1919, ['Volcano Name', 'Country']]  # Volcanoes that began erupting during the First World War.

## Exercises

1. How many volcanoes are in Spain?
2. Which is further north, Cotopaxi or Kilimanjaro?

# Database-like analysis

Pandas dataframes can be manipulated in the same way as SQL-based databases.

In [ ]:
#  Perform SQL-like SELECT and ORDER BY operations
gvp_names['Elevation (m)'].sort_values(ascending=False).head(10)

In [ ]:
#  Perform SQL-like GROUP BY operations
country_counts = gvp_names.groupby('Country').size()  # Count the number of volcanoes per country
country_counts.sort_values(ascending=False).head(10)  # Top 10 most volcanically active countries

In [ ]:
#  Plot results of groupby operations
type_sizes = gvp.groupby('Primary Volcano Type')['Elevation (m)'].mean()
type_sizes.plot(kind='bar')
plt.ylabel('Mean elevation (m)')

# Guess what: submarine volcanoes have the lowest mean elevations!

In [ ]:
#  Use the pd.merge function to perform SQL-like JOINs of data tables
factbook = pd.read_csv(os.path.join(data_dir, 'factbook_extracts.csv'))
# CIA World Factbook data modified from https://www.google.com/fusiontables/DataSource?snapid=134490

gvp_factbook = pd.merge(gvp, factbook, left_on='Country', right_on='Country', how='inner')
print(gvp_factbook.columns)  # Note the extra columns

In [ ]:
#  Perform queries using data from both tables - what proportion of volcanoes are in the poorest countries?
LOW_GDP_CUTOFF = 5000  # By convention, use ALL_CAPS for constants
low_gdp_volcanoes = gvp_factbook.loc[gvp_factbook['GDP per capita ($ PPP)'] < LOW_GDP_CUTOFF]
msg = '{} of {} volcanoes are in countries with GDP per capita <${}.\n'.format(
    len(low_gdp_volcanoes),
    len(gvp),
    LOW_GDP_CUTOFF)
print(msg)

In [ ]:
#  What is the mean life expectancy in the GVP-defined regions?
gvp_factbook_regions = gvp_factbook.groupby('Region')
gvp_factbook_regions['Life expectancy at birth'].mean().sort_values(ascending=False)

### Exporting data

In [ ]:
#  A Pandas Series (single column or row) can be exported as a list
iceland_volcanoes = gvp_names.loc[gvp_names['Country'] == 'Iceland', ['Latitude', 'Longitude']] 
volcano_names = iceland_volcanoes.index.tolist()
print(volcano_names)

In [ ]:
#  Convert a column to a list
volcano_latitudes = iceland_volcanoes['Latitude'].tolist()
print(volcano_latitudes)

In [ ]:
#  Convert a row to a list
askja_lat_lon = iceland_volcanoes.loc['Askja'].tolist()
print(askja_lat_lon)

In [ ]:
#  A Pandas Dataframe (multiple columns) can be exported as a dictionary of lists
iceland_latlons = iceland_volcanoes[['Longitude', 'Latitude']]
iceland_latlons = iceland_latlons.to_dict('list')
print(iceland_latlons)

In [ ]:
#  Data can be exported to files with .to_csv, .to_excel, .to_latex commands.
#  There are options to control the format of dates and floating point numbers.
iceland_volcanoes = gvp_names.loc[gvp_names['Country'] == 'Iceland', ['Latitude', 'Longitude']]
iceland_volcanoes.to_csv('Icelandic_volcano_locations.csv', float_format='%.2f')
# Check the working directory to see the file

# Exercises

1. Export a csv file with a list of the names and locations of stratovolcanoes over 2500 m in South America.
2. What proportion of countries list volcanoes as a natural hazard?  (hint .str.contains('volcan.*')